# **Package imports**


**Installing dependencies**

In [88]:
import sys
!{sys.executable} -m pip install category_encoders
!{sys.executable} -m pip install dython

from warnings import filterwarnings
filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
#general
import numpy as np
import pandas as pd
import category_encoders as ce
from dython.nominal import associations

#useful modules
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, matthews_corrcoef as MCC
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier

# **Data preparation & Statistics**

In [90]:
df = pd.read_csv('Dataset-vf.csv')

X = df[df.columns[:-1]]
Y = df["Y"]


**Data Statistics**

In [91]:
print("The dataset has {} samples and {} features per sample\n".format(df.shape[0], df.shape[1]-1))

print("The dataset has {} classes and they are as follow: {}\n".format(len(df['Y'].unique()), df['Y'].unique()))

class_labels, class_counts = np.unique(df['Y'], return_counts=True)
print("Dataset distribution:")
print(dict(zip(class_labels, class_counts)))
print('')

def countMissingVals(X):
  for col in X.columns:
      nanCount = X[f'{col}'].isnull().sum()
      print("Column: {} has this number of missing values = {}".format(col, nanCount))

countMissingVals(df)
print('')
def dataStatistics(X):
  for col in X.columns:
    if X[col].dtype == "object":
      pass
    else:
      print("""Feature: {} Stats, 
      Mean = {}, 
      Std = {}, 
      Max = {}, 
      Min = {}""".format(col, round(df[col].mean(), 2), round(df[col].std(), 2), df[col].max(), df[col].min()))
      
dataStatistics(X)

The dataset has 6374 samples and 15 features per sample

The dataset has 7 classes and they are as follow: ['Melon' 'Carrot' 'Orange' 'Apple' 'Berry' 'Mango' 'Peach']

Dataset distribution:
{'Apple': 1275, 'Berry': 1250, 'Carrot': 1620, 'Mango': 250, 'Melon': 1571, 'Orange': 100, 'Peach': 308}

Column: X1 has this number of missing values = 0
Column: X2 has this number of missing values = 627
Column: X3 has this number of missing values = 0
Column: X4 has this number of missing values = 0
Column: X5 has this number of missing values = 0
Column: X6 has this number of missing values = 0
Column: X7 has this number of missing values = 0
Column: X8 has this number of missing values = 0
Column: X9 has this number of missing values = 0
Column: X10 has this number of missing values = 0
Column: X11 has this number of missing values = 0
Column: X12 has this number of missing values = 0
Column: X13 has this number of missing values = 0
Column: X14 has this number of missing values = 0
Column: X15

**Categorical data Statistics**

In [92]:
X = X.drop("X15", axis=1)

x13 = X["X13"].unique()
x14 = X["X14"].unique()

print("Num of values = {}, {}".format(len(x13), x13))
print("Num of values = {}, {}".format(len(x14), x14))

Num of values = 4, [' FC11' ' FC14' ' FC13' ' FC12']
Num of values = 32, [' CA49' ' CA50' ' CA26' ' CA32' ' CA38' ' CA42' ' CA37' ' CA23' ' CA24'
 ' CA21' ' CA43' ' CA22' ' CA36' ' CA33' ' CA44' ' CA59' ' CA30' ' CA40'
 ' CA51' ' CA52' ' CA31' ' CA58' ' CA25' ' CA39' ' CA53' ' CA60' ' CA46'
 ' CA54' ' CA34' ' CA41' ' CA48' ' CA55']


**Data correlation analysis with categorical data useing Dython library**

In [93]:
complete_correlation = associations(X, nominal_columns='auto', numerical_columns=None, 
             mark_columns=False, nom_nom_assoc='cramer', num_num_assoc='pearson', 
             cramers_v_bias_correction=True, nan_strategy='replace', nan_replace_value=0, 
             ax=None, figsize=None, annot=True, fmt='.2f', cmap=None, sv_color='silver', cbar=True, vmax=1.0, 
             vmin=None, plot=True, compute_only=True, clustering=False)

df_complete_corr = complete_correlation['corr']
df_complete_corr

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14
X1,1.000000,0.004343,-0.333203,0.329017,-0.002311,0.606769,0.072518,0.229464,0.122600,0.463866,0.001618,-0.002352,0.874781,0.900729
X2,0.004343,1.000000,-0.006776,0.015085,0.014722,0.071822,-0.508053,0.293633,0.542223,-0.045059,-0.021234,-0.007682,0.156590,0.247661
X3,-0.333203,-0.006776,1.000000,0.018341,0.363640,-0.327663,-0.134366,-0.656795,-0.401927,-0.235959,0.010468,0.005183,0.305414,0.620458
X4,0.329017,0.015085,0.018341,1.000000,0.624466,0.130159,-0.019646,0.047840,0.038178,0.115299,-0.005063,-0.004866,0.255371,0.404052
X5,-0.002311,0.014722,0.363640,0.624466,1.000000,-0.113061,-0.050173,-0.213299,-0.131816,-0.089477,-0.004616,-0.009564,0.137424,0.408334
X6,0.606769,0.071822,-0.327663,0.130159,-0.113061,1.000000,-0.024238,0.267689,0.211251,0.445928,-0.002762,-0.005791,0.522569,0.634081
X7,0.072518,-0.508053,-0.134366,-0.019646,-0.050173,-0.024238,1.000000,-0.039172,-0.776999,0.064237,0.011532,0.016630,0.144763,0.557979
X8,0.229464,0.293633,-0.656795,0.047840,-0.213299,0.267689,-0.039172,1.000000,0.637684,0.122884,-0.034126,0.001968,0.282122,0.488182
X9,0.122600,0.542223,-0.401927,0.038178,-0.131816,0.211251,-0.776999,0.637684,1.000000,0.054301,-0.028194,-0.011236,0.216832,0.547694
X10,0.463866,-0.045059,-0.235959,0.115299,-0.089477,0.445928,0.064237,0.122884,0.054301,1.000000,0.012534,-0.004848,0.566034,0.633078


**Encoding Categorical data using One Hot encoding & Binary encoding**

In [94]:
def dealWithCatData(X, cols, method="OneHot"):
  if method == "OneHot":
    newX = X.copy()
    newX = pd.get_dummies(newX,columns=cols)
    return newX
  else:
    newX = X.copy()
    encoder = ce.BinaryEncoder(cols=cols)
    result = encoder.fit_transform(newX[cols])
    newX = newX.drop(cols, axis=1)
    newX = pd.concat([newX, result], axis=1)
    return newX
X = dealWithCatData(X, ["X13"], method="OneHot")
X = dealWithCatData(X, ["X14"], method="Binary")
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X13_ FC11,X13_ FC12,X13_ FC13,X13_ FC14,X14_0,X14_1,X14_2,X14_3,X14_4,X14_5
0,2709,59.0,11,120,43,150,228,217,120,2114,...,1,0,0,0,0,0,0,0,0,1
1,2802,54.0,7,0,0,693,224,225,136,162,...,1,0,0,0,0,0,0,0,1,0
2,2325,28.0,12,0,0,1260,215,213,133,162,...,0,0,0,1,0,0,0,0,1,1
3,2837,112.0,8,272,16,3649,235,231,128,6221,...,1,0,0,0,0,0,0,1,0,0
4,2509,59.0,7,134,10,900,226,226,134,5184,...,1,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6369,3101,67.0,6,170,1,824,227,228,135,1657,...,1,0,0,0,0,0,1,0,1,1
6370,3050,139.0,13,30,-3,3127,240,235,123,5770,...,1,0,0,0,0,1,0,0,1,0
6371,2080,106.0,24,30,12,582,252,202,64,108,...,0,0,0,1,0,0,1,0,0,0
6372,3011,108.0,10,212,36,2912,237,229,122,6632,...,1,0,0,0,0,0,0,0,0,1


**Helper methods to deal with missing data**

In [95]:
def replaceWithZeros(X, nan_cols):
  clean_X_Z = X.copy()
  for col in nan_cols:
    clean_X_Z[col] = clean_X_Z[col].fillna(0)
  return clean_X_Z

def replaceWithMean(X, nan_cols):
  clean_X_M = X.copy()
  for col in nan_cols:
    clean_X_M[col] = clean_X_M[col].fillna(round(clean_X_M[col].mean(), 2))
  return clean_X_M

def replaceWithDrop(X, nan_cols, drop="col"):

  if drop == "col":
    clean_X_DC = X.copy()
    for col in nan_cols:
      clean_X_DC = clean_X_DC.drop(col, axis=1)
    return clean_X_DC

  else:
    clean_X_DR = X.copy()
    clean_X_DR = clean_X_DR.dropna()
    return clean_X_DR

**Dealing with missing data**

In [96]:
def cleanMissingData(X):
  nan_cols = []
  for col in X.columns:
      nanCount = X[f'{col}'].isnull().sum()
      if nanCount > 0:
        nan_cols.append(col)

  if len(nan_cols) > 0:
    clean_X_Z = replaceWithZeros(X, nan_cols)             # clean_X_Z = a data frame cleaned by replacing the missing data by Zeros
    clean_X_M = replaceWithMean(X, nan_cols)              # clean_X_M = a data frame cleaned by replacing the missing data by Mean
    clean_X_DC = replaceWithDrop(X, nan_cols, drop="col") # clean_X_DC = a data frame cleaned by Droping the Column with missing data
    clean_X_DR = replaceWithDrop(X, nan_cols, drop="Row") # clean_X_DR = a data frame cleaned by Droping the Rows with missing data

  return clean_X_Z, clean_X_M, clean_X_DC, clean_X_DR
clean_X_Z, clean_X_M, clean_X_DC, clean_X_DR = cleanMissingData(X)

**Data Scaling**

In [97]:
def scaleData(scaler, X_train, X_test, X_val):
    scaler.fit(X_train)
    scaler.transform(X_train)
    scaler.transform(X_test)
    scaler.transform(X_val)

def scaleAll(X_train, X_test, X_val, scaler="MinMax"):
    scaleData(MinMaxScaler(),X_train, X_test, X_val) if scaler == "MinMax" else scaleData(StandardScaler(),X_train, X_test, X_val)

#### **Feature expansion:**

In [98]:
def expandFeatures(X_train, X_test, X_val, degree_val=2):
  poly = PolynomialFeatures(degree_val, include_bias=False)
  X_train_poly = poly.fit_transform(X_train) 
  X_test_poly = poly.fit_transform(X_test)
  X_val_poly = poly.fit_transform(X_val)
  return X_train_poly, X_test_poly, X_val_poly

**Splitting Data into Training (70%) & Validation (15%) & Testing (15%)**

In [99]:
X_train, X_test, Y_train, Y_test = train_test_split(clean_X_Z, Y, test_size =0.3, random_state=0, stratify=Y)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size =0.5, random_state=0, stratify=Y_test)

print(" {} features per sample before expand Features\n".format(X_train.shape[1]))
d0 = X_train.shape[1]
# X_train, X_test, X_val = expandFeatures(X_train, X_test, X_val, degree_val=3) #might slow the training process depending on the classifier (with a slight improvement on performance).
d1 = (X_train.shape[1])
print(" {} features per sample after expand Features\n".format(X_train.shape[1]))
scaleAll(X_train, X_test, X_val, scaler="Standard")

ratio = d0/d1

 22 features per sample before expand Features

 22 features per sample after expand Features



## **Unsupervised learning**

#### **Dimensionality Reduction:**

In [100]:
def reduceX(X,ratio, min):
    numberOfFeatures = X.shape[1]
    n_components = round(ratio * numberOfFeatures)
    n_components = max(n_components, min)
    pca = PCA(n_components=n_components, whiten=True)
    X = pca.fit(X).transform(X)
    return X

def reduceAll(X_train, X_test, X_val, ratio=0.25, min = 2):
    return reduceX(X_train, ratio, min), reduceX(X_test, ratio, min), reduceX(X_val, ratio, min)

**reducing number of features using PCA**

In [101]:
#it did a great job reducing the feautures but we decided to disable it since it drops the performance significantly 

# print(f"Number of features before the reduction = {X_train.shape[1]}")
# X_train, X_test, X_val = reduceAll(X_train, X_test, X_val, ratio=ratio)
# print(f"Number of features after the reduction = {X_train.shape[1]}")

# **classification:**
we will use the following classifiers in order:

**1- Linear Classifiers:**

    1. Logistic Regression.
    2. SVM.

**2- Non-Linear Classifiers:**

    1. KNN.
    2. Neural Network.
    3. Decision Tree.

**3- Ensembel Learning:**

    1. Bagging.
    2. Adaboost.
    3. Random Forest.

#### **helper functions**

In [130]:
def Gmean(testy,y_pred):
    precision = precision_score(testy, y_pred, average='micro')
    recall = recall_score(testy, y_pred, average='micro')
    return np.product(np.power([precision, recall], [0.6,0.4])) #60% precision, 40% recall
    

def testModel(model, testx, testy):
    measures = {}
    y_pred = model.predict(testx)
    measures["accuracy"] = classification_report(testy,y_pred,output_dict=True)["accuracy"]
    measures["f1_macro"] = f1_score(testy,y_pred, average="macro")
    measures["f1_weighted"] = f1_score(testy,y_pred, average="weighted")
    measures["mcc"]= MCC(testy,y_pred)
    measures["gmean"]= Gmean(testy,y_pred)
    return measures

def classify(model, trainx ,trainy, testx, testy):
    model.fit(trainx, trainy)
    return testModel(model, testx, testy)

def displayResults(models:list,X_train, Y_train, X_val, Y_val, X_test, Y_test):
    for measure in models:
        result = testModel(models[measure]["model"], X_train, Y_train)[measure]
        print(f"{measure} on Training set= {round(result*100,2)}%")

        result = testModel(models[measure]["model"], X_val, Y_val)[measure]
        print(f"{measure} on validation set= {round(result*100,2)}%")

        result = testModel(models[measure]["model"], X_test, Y_test)[measure]
        print(f"{measure} on test set= {round(result*100,2)}%\n")

def printModels(models:dict):
    if type(models) is not dict:
        print(f"{models}", end=", ")
    else:
        for model in models: 
            print(f"{model}: ",end = "")
            printModels(models[model])
        print()

## **Linear classifiers**

### **1-Logistic Regression**

In [131]:
def initializeLogisticRegression():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "C":0,
                "penalty":0
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "C":0,
                "penalty":0
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "C":0,
                "penalty":0
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "C":0,
                "penalty":0
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "C":0,
                "penalty":0
            }
        },
    }

def updateLogisticRegression(measures, bestModels, model, C, penalty):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["C"] = C
            bestModels[measureType]["hyperparameters"]["penalty"] = penalty
    return bestModels

def bestLogisticRegression(trainx, trainy, testx, testy, regularizations:list, CValues:list, num_iterations:int=1000):
    bestModels = initializeLogisticRegression()
    for penalty in regularizations:
        for C in CValues:
            model = LogisticRegression(penalty = penalty, C=C, max_iter = num_iterations, solver="lbfgs")
            measures =  classify(model, trainx, trainy, testx, testy)
            bestModels = updateLogisticRegression(measures, bestModels, model, C, penalty)

    return bestModels

**Training model:**

In [132]:
regularizations = ['l2', 'none']
cValues = [0.0001, 0.001, 0.01,0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20]
iterations = 1000

models = bestLogisticRegression(X_train, Y_train, X_val, Y_val, regularizations, cValues, iterations)
printModels(models)

accuracy: model: LogisticRegression(C=15, max_iter=1000), measure: 0.7510460251046025, hyperparameters: C: 15, penalty: l2, 

f1_macro: model: LogisticRegression(C=6, max_iter=1000), measure: 0.5536878326233475, hyperparameters: C: 6, penalty: l2, 

f1_weighted: model: LogisticRegression(C=15, max_iter=1000), measure: 0.727310899351831, hyperparameters: C: 15, penalty: l2, 

mcc: model: LogisticRegression(C=15, max_iter=1000), measure: 0.6829000610929165, hyperparameters: C: 15, penalty: l2, 

gmean: model: LogisticRegression(C=15, max_iter=1000), measure: 0.7510460251046024, hyperparameters: C: 15, penalty: l2, 




**Testing model:**

In [133]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 75.1%
accuracy on validation set= 75.1%
accuracy on test set= 74.09%

f1_macro on Training set= 56.56%
f1_macro on validation set= 55.37%
f1_macro on test set= 54.72%

f1_weighted on Training set= 72.5%
f1_weighted on validation set= 72.73%
f1_weighted on test set= 71.48%

mcc on Training set= 68.29%
mcc on validation set= 68.29%
mcc on test set= 66.97%

gmean on Training set= 75.1%
gmean on validation set= 75.1%
gmean on test set= 74.09%



### **2-SVM**

In [134]:
def initializeSVM():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "kernel":"",
                "C":0,
                "degree":None,
                "coef":None
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "kernel":"",
                "C":0,
                "degree":None,
                "coef":None
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "kernel":"",
                "C":0,
                "degree":None,
                "coef":None
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "kernel":"",
                "C":0,
                "degree":None,
                "coef":None
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "kernel":"",
                "C":0,
                "degree":None,
                "coef":None
            }
        },
    }

def updateSVM(measures, bestModels, model, C, kernel, degree=None, coefficient=None):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["C"] = C
            bestModels[measureType]["hyperparameters"]["kernel"] = kernel
            if degree is not None:
                bestModels[measureType]["hyperparameters"]["degree"] = degree
            if coefficient is not None:
                bestModels[measureType]["hyperparameters"]["coef"] = coefficient
    return bestModels

def bestSVM(trainx, trainy, testx, testy, CValues:list, kernels:list, degrees:list=None, coefficients:list=None):
    bestModels = initializeSVM()
    for C in CValues:
        for kernel in kernels:
            if kernel == "sigmoid":
                for coefficient in coefficients:
                    model = SVC(C = C, kernel = kernel, coef0 = coefficient)
                    measures =  classify(model, trainx, trainy, testx, testy)
                    bestModels = updateSVM(measures, bestModels, model, C, kernel, coefficient=coefficient)
            elif kernel == "poly":
                for degree in degrees:
                    for coefficient in coefficients:
                        model = SVC(C = C, kernel = kernel, degree = degree, coef0=coefficient)
                        measures =  classify(model, trainx, trainy, testx, testy)
                        bestModels = updateSVM(measures, bestModels, model, C, kernel, degree=degree, coefficient=coefficient)
            else:
                model = SVC(C = C, kernel = kernel)
                measures =  classify(model, trainx, trainy, testx, testy)
                bestModels = updateSVM(measures, bestModels, model, C, kernel)
    return bestModels

**Training model:**

In [107]:
kernels = ['linear','sigmoid','rbf','poly']
CValues = [0.01, 0.1, 0.5]
degrees = [2,3,4]
coefficients = [1]

models = bestSVM(X_train, Y_train, X_val, Y_val, CValues, kernels, degrees, coefficients)
printModels(models)

accuracy: model: SVC(C=0.1, kernel='linear'), measure: 0.8221757322175732, hyperparameters: kernel: linear, C: 0.1, degree: None, coef: None, 

f1_macro: model: SVC(C=0.01, kernel='linear'), measure: 0.7187445031746336, hyperparameters: kernel: linear, C: 0.01, degree: None, coef: None, 

f1_weighted: model: SVC(C=0.1, kernel='linear'), measure: 0.8197784443746438, hyperparameters: kernel: linear, C: 0.1, degree: None, coef: None, 

mcc: model: SVC(C=0.1, kernel='linear'), measure: 0.7746480699936555, hyperparameters: kernel: linear, C: 0.1, degree: None, coef: None, 

gmean: model: SVC(C=0.1, kernel='linear'), measure: 0.8221757322175731, hyperparameters: kernel: linear, C: 0.1, degree: None, coef: None, 




**Testing model:**

In [108]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 82.4%
accuracy on validation set= 82.22%
accuracy on test set= 81.5%

f1_macro on Training set= 71.99%
f1_macro on validation set= 71.87%
f1_macro on test set= 68.92%

f1_weighted on Training set= 81.97%
f1_weighted on validation set= 81.98%
f1_weighted on test set= 81.11%

mcc on Training set= 77.67%
mcc on validation set= 77.46%
mcc on test set= 76.53%

gmean on Training set= 82.4%
gmean on validation set= 82.22%
gmean on test set= 81.5%



## **Non-Linear classifiers**

### **1-KNN**

In [109]:
def initializeKNN():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "kRange":0
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "kRange":0
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "kRange":0
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "kRange":0
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "kRange":0
            }
        }
    }

def updateKNN(measures, bestModels, model, k, algorithm):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["kRange"] = k
            bestModels[measureType]["hyperparameters"]["algorithm"] = algorithm
    return bestModels

def bestKNN(trainx, trainy, testx, testy, algorithms:list, kRange:list):
    bestModels = initializeKNN()
    for algorithm in algorithms:
        for k in kRange:
            model = KNeighborsClassifier(n_neighbors=k, algorithm = algorithm)
            measures =  classify(model, trainx, trainy, testx, testy)
            bestModels = updateKNN(measures, bestModels, model, k, algorithm)
    return bestModels

**Training model:**

In [110]:
algorithms = ['auto', 'ball_tree', 'kd_tree', 'brute']
kRange = [1,2,3,4,5,6,7,8,9,10,12,15]

models = bestKNN(X_train, Y_train, X_val, Y_val, algorithms, kRange)
printModels(models)

accuracy: model: KNeighborsClassifier(n_neighbors=3), measure: 0.8096234309623431, hyperparameters: algorithm: auto, kRange: 3, 

f1_macro: model: KNeighborsClassifier(n_neighbors=7), measure: 0.7506002778669648, hyperparameters: algorithm: auto, kRange: 7, 

f1_weighted: model: KNeighborsClassifier(n_neighbors=4), measure: 0.8042937444687809, hyperparameters: algorithm: auto, kRange: 4, 

mcc: model: KNeighborsClassifier(n_neighbors=3), measure: 0.7601134803907709, hyperparameters: algorithm: auto, kRange: 3, 

gmean: model: KNeighborsClassifier(n_neighbors=3), measure: 0.8096234309623431, hyperparameters: algorithm: auto, kRange: 3, 




**Testing model:**

In [111]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 89.89%
accuracy on validation set= 80.96%
accuracy on test set= 80.36%

f1_macro on Training set= 79.81%
f1_macro on validation set= 75.06%
f1_macro on test set= 73.28%

f1_weighted on Training set= 88.73%
f1_weighted on validation set= 80.43%
f1_weighted on test set= 80.11%

mcc on Training set= 87.28%
mcc on validation set= 76.01%
mcc on test set= 75.2%

gmean on Training set= 89.89%
gmean on validation set= 80.96%
gmean on test set= 80.36%



### **2-Neural Network**

In [112]:
def initializeNeuralNetwork():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "numOfHiddenLayers":0,
                "activation":"",
                "tol":0,
                "learning_rate":0,
                "max_iterations":0,
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "numOfHiddenLayers":0,
                "activation":"",
                "tol":0,
                "learning_rate":0,
                "max_iterations":0,
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "numOfHiddenLayers":0,
                "activation":"",
                "tol":0,
                "learning_rate":0,
                "max_iterations":0,
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "numOfHiddenLayers":0,
                "activation":"",
                "tol":0,
                "learning_rate":0,
                "max_iterations":0,
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "numOfHiddenLayers":0,
                "activation":"",
                "tol":0,
                "learning_rate":0,
                "max_iterations":0,
            }
        }
    }

def updateDecisionTree(measures, bestModels, model, hiddenLayers, activation, tol, learningRate, max_iteration):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["numOfHiddenLayers"] = hiddenLayers
            bestModels[measureType]["hyperparameters"]["activation"] = activation
            bestModels[measureType]["hyperparameters"]["tol"] = tol
            bestModels[measureType]["hyperparameters"]["learning_rate"] = learningRate
            bestModels[measureType]["hyperparameters"]["max_iterations"] = max_iteration
    return bestModels

def bestNeuralNetwork(trainx, trainy, testx, testy, numberOfHiddenLayers:list, activations:list, tols:list, learningRates:list, max_iterations:list):
    bestModels = initializeNeuralNetwork()
    for activation in activations:
        for tol in tols:
            for hiddenLayers in numberOfHiddenLayers:
                for learningRate in learningRates:
                    for max_iteration in max_iterations:
                        model = MLPClassifier(hidden_layer_sizes = hiddenLayers, learning_rate = "constant", learning_rate_init = learningRate, max_iter = max_iteration, tol = tol)
                        measures =  classify(model, trainx, trainy, testx, testy)
                        bestModels = updateDecisionTree(measures, bestModels, model, hiddenLayers, activation, tol, learningRate, max_iteration)
    return bestModels

**Training model:**

In [113]:
numOfHiddenLayers = [1,2,3]
activations = ["relu", "logistic", "tanh"]
tols = [1e-4, 1e-5]
learning_rates = [0.0001, 0.0005, 0.001, 0.005, 0.01]
max_iterations = [100,1000,10000]

models = bestNeuralNetwork(X_train, Y_train, X_val, Y_val, numOfHiddenLayers, activations, tols, learning_rates, max_iterations)
printModels(models)

accuracy: model: MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.005, max_iter=1000,
              tol=1e-05), measure: 0.7311715481171548, hyperparameters: numOfHiddenLayers: 3, activation: relu, tol: 1e-05, learning_rate: 0.005, max_iterations: 1000, 

f1_macro: model: MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.005, max_iter=1000,
              tol=1e-05), measure: 0.42630171218522817, hyperparameters: numOfHiddenLayers: 3, activation: relu, tol: 1e-05, learning_rate: 0.005, max_iterations: 1000, 

f1_weighted: model: MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.005, max_iter=1000,
              tol=1e-05), measure: 0.6771736362683463, hyperparameters: numOfHiddenLayers: 3, activation: relu, tol: 1e-05, learning_rate: 0.005, max_iterations: 1000, 

mcc: model: MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.005, max_iter=1000,
              tol=1e-05), measure: 0.6623565914699655, hyperparameters: numOfHiddenLayers: 3, activation: relu, tol

**Testing model:**

In [114]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 72.63%
accuracy on validation set= 73.12%
accuracy on test set= 71.37%

f1_macro on Training set= 42.21%
f1_macro on validation set= 42.63%
f1_macro on test set= 41.37%

f1_weighted on Training set= 67.07%
f1_weighted on validation set= 67.72%
f1_weighted on test set= 65.85%

mcc on Training set= 65.76%
mcc on validation set= 66.24%
mcc on test set= 64.02%

gmean on Training set= 72.63%
gmean on validation set= 73.12%
gmean on test set= 71.37%



### **3-Decision Tree**

In [115]:
def initializeDecisionTree():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "splitter":"",
                "max_depth":0,
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "splitter":"",
                "max_depth":0,
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "splitter":"",
                "max_depth":0,
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "splitter":"",
                "max_depth":0,
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "splitter":"",
                "max_depth":0,
            }
        },

    }

def updateDecisionTree(measures, bestModels, model, criterion, splitter, max_depth):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["criterion"] = criterion
            bestModels[measureType]["hyperparameters"]["splitter"] = splitter
            bestModels[measureType]["hyperparameters"]["max_depth"] = max_depth
    return bestModels

def bestDecisionTree(trainx, trainy, testx, testy, criterions:list, splitters:list, max_depths:list):
    bestModels = initializeDecisionTree()
    for criterion in criterions:
        for splitter in splitters:
            for max_depth in max_depths:
                model = DecisionTreeClassifier(criterion=criterion, splitter=splitter, max_depth=max_depth)
                measures =  classify(model, trainx, trainy, testx, testy)
                bestModels = updateDecisionTree(measures, bestModels, model, criterion, splitter, max_depth)
    return bestModels

**Training model:**

In [116]:
criterions = ["gini","entropy"]
splitters = ["best","random"]
max_depths = [None, 100, 200, 500, 1000, 2000]

models = bestDecisionTree(X_train, Y_train, X_val, Y_val, criterions, splitters, max_depths)
printModels(models)

accuracy: model: DecisionTreeClassifier(criterion='entropy', max_depth=2000, splitter='random'), measure: 0.9027196652719666, hyperparameters: criterion: entropy, splitter: random, max_depth: 2000, 

f1_macro: model: DecisionTreeClassifier(max_depth=200, splitter='random'), measure: 0.8526030467353168, hyperparameters: criterion: gini, splitter: random, max_depth: 200, 

f1_weighted: model: DecisionTreeClassifier(criterion='entropy', max_depth=2000, splitter='random'), measure: 0.9029755542685424, hyperparameters: criterion: entropy, splitter: random, max_depth: 2000, 

mcc: model: DecisionTreeClassifier(criterion='entropy', max_depth=2000, splitter='random'), measure: 0.8774138645824169, hyperparameters: criterion: entropy, splitter: random, max_depth: 2000, 

gmean: model: DecisionTreeClassifier(criterion='entropy', max_depth=2000, splitter='random'), measure: 0.9027196652719665, hyperparameters: criterion: entropy, splitter: random, max_depth: 2000, 




**Testing model:**

In [117]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 100.0%
accuracy on validation set= 90.27%
accuracy on test set= 89.13%

f1_macro on Training set= 100.0%
f1_macro on validation set= 85.26%
f1_macro on test set= 84.7%

f1_weighted on Training set= 100.0%
f1_weighted on validation set= 90.3%
f1_weighted on test set= 89.08%

mcc on Training set= 100.0%
mcc on validation set= 87.74%
mcc on test set= 86.28%

gmean on Training set= 100.0%
gmean on validation set= 90.27%
gmean on test set= 89.13%



## **Ensembel Learning**

### **1-Bagging**

In [118]:
def initializeBagging():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "max_features":0,
                "estimators":0,
                "oob_score":False,
                "max_samples":0
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "max_features":0,
                "estimators":0,
                "oob_score":False,
                "max_samples":0
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "max_features":0,
                "estimators":0,
                "oob_score":False,
                "max_samples":0
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "max_features":0,
                "estimators":0,
                "oob_score":False,
                "max_samples":0
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "max_features":0,
                "estimators":0,
                "oob_score":False,
                "max_samples":0
            }
        }
    }

def updateBagging(measures, bestModels, model, estimators, max_samples, max_features, oob_score):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["max_features"] = max_features
            bestModels[measureType]["hyperparameters"]["estimators"] = estimators
            bestModels[measureType]["hyperparameters"]["max_samples"] = max_samples
            bestModels[measureType]["hyperparameters"]["oob_score"] = oob_score
    return bestModels

def bestBagging(trainx, trainy, testx, testy, estimators:list, max_samples_range:list, max_features_range:list, oob_scores:list):
    bestModels = initializeBagging()
    for max_samples in max_samples_range:
        for max_features in max_features_range:
            for estimator in estimators:
                for oob_score in oob_scores:
                    model = BaggingClassifier(n_estimators=estimator, max_samples=max_samples, max_features=max_features, oob_score = oob_score)
                    measures =  classify(model, trainx, trainy, testx, testy)
                    bestModels = updateBagging(measures, bestModels, model, estimator, max_samples, max_features, oob_scores)
    return bestModels


**Training model:**

In [119]:
estimators = [50,70,80,90,100,110,120,150,200]
oob_scores = [True, False]
max_samples = [1, 100, 150, 200, 500, 1000, 2000]
max_features = [1,2,3,4,5,10,15,20]

models = bestBagging(X_train, Y_train, X_val, Y_val, estimators, max_samples, max_features, max_depths)
printModels(models)

accuracy: model: BaggingClassifier(max_features=15, max_samples=2000, n_estimators=80,
                  oob_score=200), measure: 0.9236401673640168, hyperparameters: max_features: 15, estimators: 80, oob_score: [None, 100, 200, 500, 1000, 2000], max_samples: 2000, 

f1_macro: model: BaggingClassifier(max_features=20, max_samples=2000, n_estimators=150,
                  oob_score=100), measure: 0.886206507777847, hyperparameters: max_features: 20, estimators: 150, oob_score: [None, 100, 200, 500, 1000, 2000], max_samples: 2000, 

f1_weighted: model: BaggingClassifier(max_features=15, max_samples=2000, n_estimators=80,
                  oob_score=200), measure: 0.9218921471896853, hyperparameters: max_features: 15, estimators: 80, oob_score: [None, 100, 200, 500, 1000, 2000], max_samples: 2000, 

mcc: model: BaggingClassifier(max_features=15, max_samples=2000, n_estimators=80,
                  oob_score=200), measure: 0.9033254369778683, hyperparameters: max_features: 15, estimators: 

**Testing model:**

In [120]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 98.5%
accuracy on validation set= 92.36%
accuracy on test set= 93.42%

f1_macro on Training set= 96.93%
f1_macro on validation set= 88.62%
f1_macro on test set= 88.62%

f1_weighted on Training set= 98.49%
f1_weighted on validation set= 92.19%
f1_weighted on test set= 93.29%

mcc on Training set= 98.1%
mcc on validation set= 90.33%
mcc on test set= 91.67%

gmean on Training set= 98.5%
gmean on validation set= 92.36%
gmean on test set= 93.42%



### **2-Adaboost**

In [121]:
def initializeAdaboost():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "estimator":0,
                "alpha":0
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "estimator":0,
                "alpha":0
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "algorithm":"",
                "alpha":0
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "estimator":0,
                "alpha":0
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "estimator":0,
                "alpha":0
            }
        }
    }

def updateAdaboost(measures, bestModels, model, estimator, alpha):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["estimator"] = estimator
            bestModels[measureType]["hyperparameters"]["alpha"] = alpha
    return bestModels

def bestAdaboost(trainx, trainy, testx, testy, estimators:list, alphas:list):
    bestModels = initializeAdaboost()
    for estimator in estimators:
        for alpha in alphas:
            model = AdaBoostClassifier(n_estimators=estimator, learning_rate=alpha)
            measures =  classify(model, trainx, trainy, testx, testy)
            bestModels = updateAdaboost(measures, bestModels, model, estimator, alpha)
    return bestModels

**Training model:**

In [122]:
alphas = [0.001,0.01,0.1,0.25,0.5,0.75,1,1.5,2]
estimators = [50,60,65,70,75,80,90,100]

models = bestAdaboost(X_train, Y_train, X_val, Y_val, estimators, alphas)
printModels(models)

accuracy: model: AdaBoostClassifier(learning_rate=0.001), measure: 0.6684100418410042, hyperparameters: estimator: 50, alpha: 0.001, 

f1_macro: model: AdaBoostClassifier(learning_rate=2, n_estimators=70), measure: 0.42114567608455467, hyperparameters: estimator: 70, alpha: 2, 

f1_weighted: model: AdaBoostClassifier(learning_rate=0.001), measure: 0.551787142294103, hyperparameters: algorithm: , alpha: 0.001, estimator: 50, 

mcc: model: AdaBoostClassifier(learning_rate=0.001), measure: 0.599289551634668, hyperparameters: estimator: 50, alpha: 0.001, 

gmean: model: AdaBoostClassifier(learning_rate=0.001), measure: 0.6684100418410043, hyperparameters: estimator: 50, alpha: 0.001, 




**Testing model:**

In [123]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 67.09%
accuracy on validation set= 66.84%
accuracy on test set= 67.29%

f1_macro on Training set= 41.2%
f1_macro on validation set= 42.11%
f1_macro on test set= 38.91%

f1_weighted on Training set= 55.42%
f1_weighted on validation set= 55.18%
f1_weighted on test set= 55.67%

mcc on Training set= 60.3%
mcc on validation set= 59.93%
mcc on test set= 60.62%

gmean on Training set= 67.09%
gmean on validation set= 66.84%
gmean on test set= 67.29%



### **3-Random Forest**

In [124]:
def initializeRandomForest():
    return{
        "accuracy":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "estimators":0,
                "max_depth":0,
            }
        },
        "f1_macro":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "estimators":0,
                "max_depth":0,
            }
        },
        "f1_weighted":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "estimators":0,
                "max_depth":0,
            }
        },
        "mcc":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "estimators":0,
                "max_depth":0,
            }
        },
        "gmean":{
            "model":None,
            "measure":0,
            "hyperparameters":{
                "criterion":"",
                "estimators":0,
                "max_depth":0,
            }
        }
    }

def updateRandomForest(measures, bestModels, model, criterion, estimators, max_depth):
    for measureType in measures:
        if (measures[measureType]> bestModels[measureType]["measure"]):
            bestModels[measureType]["measure"] = measures[measureType]
            bestModels[measureType]["model"] = model
            bestModels[measureType]["hyperparameters"]["criterion"] = criterion
            bestModels[measureType]["hyperparameters"]["estimators"] = estimators
            bestModels[measureType]["hyperparameters"]["max_depth"] = max_depth
    return bestModels

def bestRandomForest(trainx, trainy, testx, testy, criterions:list, estimators:list, max_depths:list):
    bestModels = initializeRandomForest()
    for criterion in criterions:
        for estimator in estimators:
            for max_depth in max_depths:
                model = RandomForestClassifier(criterion=criterion, n_estimators=estimator, max_depth=max_depth)
                measures =  classify(model, trainx, trainy, testx, testy)
                bestModels = updateRandomForest(measures, bestModels, model, criterion, estimator, max_depth)
    return bestModels

**Training model:**

In [125]:
criterions = ["gini","entropy"]
estimators = [50,70,80,90,100,110,120,150,200]
max_depths = [None, 100, 200, 500, 1000, 2000]

models = bestRandomForest(X_train, Y_train, X_val, Y_val, criterions, estimators, max_depths)
printModels(models)

accuracy: model: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=70), measure: 0.9361924686192469, hyperparameters: criterion: entropy, estimators: 70, max_depth: 100, 

f1_macro: model: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=50), measure: 0.8950690720086759, hyperparameters: criterion: entropy, estimators: 50, max_depth: 100, 

f1_weighted: model: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=70), measure: 0.9349316769139007, hyperparameters: criterion: entropy, estimators: 70, max_depth: 100, 

mcc: model: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=70), measure: 0.9192384802989317, hyperparameters: criterion: entropy, estimators: 70, max_depth: 100, 

gmean: model: RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=70), measure: 0.9361924686192469, hyperparameters: criterion: entropy, estimators: 70, max_depth: 100, 




**Testing model:**

In [126]:
displayResults(models, X_train, Y_train, X_val, Y_val, X_test, Y_test)

accuracy on Training set= 100.0%
accuracy on validation set= 93.62%
accuracy on test set= 93.31%

f1_macro on Training set= 100.0%
f1_macro on validation set= 89.51%
f1_macro on test set= 88.5%

f1_weighted on Training set= 100.0%
f1_weighted on validation set= 93.49%
f1_weighted on test set= 93.13%

mcc on Training set= 100.0%
mcc on validation set= 91.92%
mcc on test set= 91.54%

gmean on Training set= 100.0%
gmean on validation set= 93.62%
gmean on test set= 93.31%



# **Project report**

###best performing classifier was ***Random Forest*** with the following results: **93.31%** on **Accuracy** measure without **Feature Expansion**, and with **Feature Expansion** the results were **93.83%** on **Accuracy** measure, with performance diffrence of **+0.55%**, we decided that the performance increase was not worth the increase in training time of all classifiers.

###the results shows that **accuracy** was the best measure for this classifier followed by **Gmean** since it provided much similar results as compared to accuracy.

### ***Note***: Random Forest follows Decision Tree's behavior which have slight randomness while splitting data which might change the accuracy slightly with each run.

### **Feature Expansion:** might slow the training process depending on the classifier (with a slight improvement on performance).

### **Dimensionality Reduction:** it did a great job reducing the features but we decided to disable it since it drops the performance significantly.

###**Scaling Data:** we found that the **Standard Scaler** preformed slightly better.